# QuestionGeneration [🔗](https://gpt-index.readthedocs.io/en/latest/examples/evaluation/QuestionGeneration.html)
This notebook walks through the process of generating a list of questions that could be asked about your data. This is useful for setting up an evaluation pipeline using the FaithfulnessEvaluator and RelevancyEvaluator evaluation tools.

In [12]:
import nest_asyncio
nest_asyncio.apply()

In [2]:
import logging
import sys
import pandas as pd

logging.basicConfig(stream=sys.stdout, level=logging.INFO)
logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))

In [3]:
from llama_index.evaluation import DatasetGenerator, RelevancyEvaluator
from llama_index import (
    SimpleDirectoryReader,
    VectorStoreIndex,
    ServiceContext,
    LLMPredictor,
    Response,
)
from llama_index.llms import OpenAI

In [7]:
reader = SimpleDirectoryReader("../data/") 
documents = reader.load_data()

In [8]:
documents

[Document(id_='d5186211-02cb-41eb-889d-59c9b2b2beb3', embedding=None, metadata={'page_label': '1', 'file_name': 'Christos2023Q4Goals+Values.pdf'}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={}, hash='2673340e6a6f92d2c5299e5ba02a3e5c41fea9fee830b5c439006fd30f9148d8', text="Christos'\n2023\nQ4\nGoals\nand\nValues\nWhat\ndo\nI\nwant\nto\naccomplish?\nI\nwant\nto\nmake\na\npositive\nimpact\non\nsociety\nby\naccelerating\ntechnological\nadvancement\nusing\nArtificial\nIntelligence\nand\nMachine\nLearning\ntools\nto\nsolve\nreal-world\nproblems.\nI\nam\nvery\npassionate\nabout\nit\nand\nenjoying\nlearning\nas\nmuch\nas\nI\ncan\non\nthe\ntopic.\nI\nlove\nhelping\npeople\nand\nlearning\nabout\ntheir\nexperiences.\nI\nam\nlooking\nfor\na\nrole\nin\nan\nenvironment\nthat\nencourages\nlearning,\ngrowth\n(subject-matter\nexpertise),\na\nteam\nthat\nis\nalso\npassionate\nabout\nwhat\nthey\ndo\nand\nideally\non\na\nmission\nto\nmake\na\npositive\nimpact\nI\nalign\n

In [10]:
data_generator = DatasetGenerator.from_documents(documents)

chunk_size_limit is deprecated, please specify chunk_size instead


In [13]:
eval_questions = data_generator.generate_questions_from_nodes()
eval_questions

INFO:openai:message='OpenAI API response' path=https://api.openai.com/v1/chat/completions processing_ms=14987 request_id=c95932769a06bdb7e07b3c047dd391bd response_code=200
message='OpenAI API response' path=https://api.openai.com/v1/chat/completions processing_ms=14987 request_id=c95932769a06bdb7e07b3c047dd391bd response_code=200
INFO:openai:message='OpenAI API response' path=https://api.openai.com/v1/chat/completions processing_ms=15236 request_id=59d947dcbf81be8d2840c70598b37792 response_code=200
message='OpenAI API response' path=https://api.openai.com/v1/chat/completions processing_ms=15236 request_id=59d947dcbf81be8d2840c70598b37792 response_code=200
INFO:openai:message='OpenAI API response' path=https://api.openai.com/v1/chat/completions processing_ms=15512 request_id=77525c5cc66b1686b459409414e56ab2 response_code=200
message='OpenAI API response' path=https://api.openai.com/v1/chat/completions processing_ms=15512 request_id=77525c5cc66b1686b459409414e56ab2 response_code=200
INFO

["What is Christos' main goal for the fourth quarter of 2023?",
 'How does Christos plan to make a positive impact on society?',
 'What is Christos passionate about and enjoying learning?',
 'What kind of environment is Christos looking for in a role?',
 'What type of company is Christos hoping to work for?',
 'What values does Christos want the company to align with?',
 'How does Christos hope to add value to a team?',
 'What industry experience does Christos want to gain in AI?',
 "What is Christos' ultimate goal in relation to AI?",
 'What does Christos hope to build one day using AI?',
 'How does the individual in the context information describe their role as a professional?',
 'How does the individual demonstrate empathy in their professional life?',
 'How does the individual show humility in their approach to their work?',
 'What are some of the passions and interests mentioned by the individual?',
 'How does the individual express their philanthropic nature through their work?'

In [18]:
# gpt-4
gpt4 = OpenAI(temperature=0, model="gpt-4")
service_context_gpt4 = ServiceContext.from_defaults(llm=gpt4)

In [19]:
evaluator_gpt4 = RelevancyEvaluator(service_context=service_context_gpt4)

In [20]:
# create vector index
vector_index = VectorStoreIndex.from_documents(
    documents, service_context=service_context_gpt4
)

In [21]:
# define jupyter display function
def display_eval_df(query: str, response: Response, eval_result: str) -> None:
    eval_df = pd.DataFrame(
        {
            "Query": query,
            "Response": str(response),
            "Source": response.source_nodes[0].node.get_content()[:1000] + "...",
            "Evaluation Result": eval_result,
        },
        index=[0],
    )
    eval_df = eval_df.style.set_properties(
        **{
            "inline-size": "600px",
            "overflow-wrap": "break-word",
        },
        subset=["Response", "Source"]
    )
    display(eval_df)

In [22]:
query_engine = vector_index.as_query_engine()
response_vector = query_engine.query(eval_questions[1])
eval_result = evaluator_gpt4.evaluate_response(
    query=eval_questions[1], response=response_vector
)

INFO:openai:message='OpenAI API response' path=https://api.openai.com/v1/chat/completions processing_ms=673 request_id=d239a82428f89b7665934139b7da61c3 response_code=200
message='OpenAI API response' path=https://api.openai.com/v1/chat/completions processing_ms=673 request_id=d239a82428f89b7665934139b7da61c3 response_code=200


In [24]:
eval_questions[1]

'How does Christos plan to make a positive impact on society?'

In [25]:
response_vector

Response(response='Christos plans to make a positive impact on society by accelerating technological advancement using Artificial Intelligence and Machine Learning tools to solve real-world problems. He is passionate about this and enjoys learning as much as he can on the topic. He is also looking for a role in an environment that encourages learning and growth, with a team that is passionate about what they do. Ideally, he wants to find a company that is focused on solving real-world problems with AI using technology and/or scientific research. His goal is to leverage his experience and strengths to add value to the team and gain a breadth and depth of industry experience in AI. This will enable him to contribute to a greater good and potentially build something of his own that solves a real-world problem through AI.', source_nodes=[NodeWithScore(node=TextNode(id_='fc39e74a-a9cc-4cad-9869-75052faec1a3', embedding=None, metadata={'page_label': '1', 'file_name': 'Christos2023Q4Goals+Val

In [29]:
eval_result

EvaluationResult(query='How does Christos plan to make a positive impact on society?', contexts=None, response='Christos plans to make a positive impact on society by accelerating technological advancement using Artificial Intelligence and Machine Learning tools to solve real-world problems. He is passionate about this and enjoys learning as much as he can on the topic. He is also looking for a role in an environment that encourages learning and growth, with a team that is passionate about what they do. Ideally, he wants to find a company that is focused on solving real-world problems with AI using technology and/or scientific research. His goal is to leverage his experience and strengths to add value to the team and gain a breadth and depth of industry experience in AI. This will enable him to contribute to a greater good and potentially build something of his own that solves a real-world problem through AI.', passing=True, feedback='YES', score=1.0)

In [30]:
display_eval_df(eval_questions[1], response_vector, eval_result.feedback)

,Query,Response,Source,Evaluation Result
0,How does Christos plan to make a positive impact on society?,"Christos plans to make a positive impact on society by accelerating technological advancement using Artificial Intelligence and Machine Learning tools to solve real-world problems. He is passionate about this and enjoys learning as much as he can on the topic. He is also looking for a role in an environment that encourages learning and growth, with a team that is passionate about what they do. Ideally, he wants to find a company that is focused on solving real-world problems with AI using technology and/or scientific research. His goal is to leverage his experience and strengths to add value to the team and gain a breadth and depth of industry experience in AI. This will enable him to contribute to a greater good and potentially build something of his own that solves a real-world problem through AI.","Christos' 2023 Q4 Goals and Values What do I want to accomplish? I want to make a positive impact on society by accelerating technological advancement using Artificial Intelligence and Machine Learning tools to solve real-world problems. I am very passionate about it and enjoying learning as much as I can on the topic. I love helping people and learning about their experiences. I am looking for a role in an environment that encourages learning, growth (subject-matter expertise), a team that is also passionate about what they do and ideally on a mission to make a positive impact I align with. I want to find a company that is focused on solving real world problems with AI using technology and/or scientific research. Ideally this company matches my values, enables my working style and encourages me to grow and learn. I want to be in a role that leverages my experience and strengths to add value to the team. I hope that the role will also allow me to gain a breadth and depth of industry ex...",YES


In [32]:
eval_result.json()

'{"query": "How does Christos plan to make a positive impact on society?", "contexts": null, "response": "Christos plans to make a positive impact on society by accelerating technological advancement using Artificial Intelligence and Machine Learning tools to solve real-world problems. He is passionate about this and enjoys learning as much as he can on the topic. He is also looking for a role in an environment that encourages learning and growth, with a team that is passionate about what they do. Ideally, he wants to find a company that is focused on solving real-world problems with AI using technology and/or scientific research. His goal is to leverage his experience and strengths to add value to the team and gain a breadth and depth of industry experience in AI. This will enable him to contribute to a greater good and potentially build something of his own that solves a real-world problem through AI.", "passing": true, "feedback": "YES", "score": 1.0}'

In [ ]:
eval_results = [evaluator_gpt4.evaluate_response(query=eval_questions[i], response=query_engine.query(eval_questions[i])) for i in range(len(eval_questions))]

In [55]:
# Convert to DataFrame
df = pd.DataFrame([eval.__dict__ for eval in eval_results])
df.head()

,query,contexts,response,passing,feedback,score
0,What is Christos' main goal for the fourth qua...,None,Christos' main goal for the fourth quarter of ...,True,YES,1.0
1,How does Christos plan to make a positive impa...,None,Christos plans to make a positive impact on so...,True,YES,1.0
2,What is Christos passionate about and enjoying...,None,Christos is passionate about making a positive...,True,YES,1.0
3,What kind of environment is Christos looking f...,None,Christos is looking for an environment that en...,True,YES,1.0
4,What type of company is Christos hoping to wor...,None,Christos is hoping to work for a company that ...,True,YES,1.0


In [57]:
df.to_csv("../output/cam-qa-dataset.csv")